# Using the SpaCy pipeline

This task is aiming to demonstrate the tokenization capabilites of [SpaCy](https://spacy.io/), as well as to serve as an introduction to the pipeline's capabilities combined with [rule based matching](https://spacy.io/usage/rule-based-matching).

Our goal will be to process the demonstration text, as well as to correct for some peculiarities, like special pronunciation marks, wide-spread abbreviations and foreign language insertions into our text.

It is mandatory, to stick to SpaCy based pipeline operations so as to make our analysis reproducible by running the pipeline on other texts presumably coming from the same corpus.

## Our demonstration text

Original from [Deutsche Sprache](https://de.wikipedia.org/wiki/Deutsche_Sprache) Wikipedia entry - with some modifications.

In [47]:
text= '''Die deutsche Sprache bzw. Deutsch ([dɔʏ̯t͡ʃ]; abgekürzt dt. oder dtsch.) ist eine westgermanische Sprache.

And this is an English sentence inbetween.

Ihr Sprachraum umfasst Deutschland, Österreich, die Deutschschweiz, Liechtenstein, Luxemburg, Ostbelgien, Südtirol, das Elsass und Lothringen sowie Nordschleswig. Außerdem ist sie eine Minderheitensprache in einigen europäischen und außereuropäischen Ländern, z. B. in Rumänien und Südafrika, sowie Nationalsprache im afrikanischen Namibia.'''

## Basic usage

After installing SpaCy, let us demonstrate it's basic usage by analysing our text.

In [48]:
%%capture
!pip install tabulate >> /dev/null
!pip install spacy

In [49]:
# We download the German language models for Spacy
!python -m spacy download de_core_news_sm

✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')


In [50]:
# Ok, we installed SpaCy, but do we have a model for German?
# Something has to be done here to get it!

import de_core_news_sm

In [51]:
# Please do the appropriate imports for SpaCy and it's rule based Matcher class!

# import spacy
import spacy

# Import the Matcher
from spacy.matcher import Matcher

# Please don't forget to instantiate the language model that we will use later on for analysis
nlp = spacy.load('de_core_news_sm')

In [52]:
# And please use the model to analyse the text from above!
doc = nlp(text)

### Helper functions for nice printout

We just define some helper functions for nice printout. Nothing to do here, except to observe the ways one can iterate over a corpus or sentence, as well as the nice output of [Tabulate](https://bitbucket.org/astanin/python-tabulate/src/master/). 

In [53]:
from tabulate import tabulate

def print_sentences(doc):
    for sentence in doc.sents:
        print(sentence,"\n")

def print_tokens_for_sentence(doc,sentence_num, stopwords=False):
    attribs=[]
    for token in list(doc.sents)[sentence_num]:
        if token.has_extension("is_lemma_stop"):
            if stopwords and token._.is_lemma_stop:
                pass
            else:
                attribs.append([token.text, token.lemma_, token.pos_])
        else:
            attribs.append([token.text, token.lemma_, token.pos_])
    print(tabulate(attribs))


In [54]:
#lets check how many sentences we have in our text
j = 0
for i in doc.sents:
  j=j+1
print('We have ' + str(j)+ ' sentences in text')  

We have 5 sentences in text


In [55]:
print_sentences(doc)

Die deutsche Sprache bzw. Deutsch ([dɔʏ̯t͡ʃ]; abgekürzt dt. 

oder dtsch.) ist eine westgermanische Sprache.

 

And this is an English sentence inbetween.

 

Ihr Sprachraum umfasst Deutschland, Österreich, die Deutschschweiz, Liechtenstein, Luxemburg, Ostbelgien, Südtirol, das Elsass und Lothringen sowie Nordschleswig. 

Außerdem ist sie eine Minderheitensprache in einigen europäischen und außereuropäischen Ländern, z. B. in Rumänien und Südafrika, sowie Nationalsprache im afrikanischen Namibia. 



In [56]:
print_tokens_for_sentence(doc,1)

---------------  ---------------  -----
oder             oder             CCONJ
dtsch            dtsch            ADJ
.                .                PUNCT
)                )                PUNCT
ist              sein             AUX
eine             einen            DET
westgermanische  westgermanische  ADJ
Sprache          Sprache          NOUN
.                .                PUNCT
                                  SPACE
---------------  ---------------  -----


## Matching "zum Beispiel"

We are a bit frustrated, that the standard analysis pipeline does not know, that in German, "z. B." is the abbreviation of "zum Beispiel" (like eg. is for "for example"), thus we would like to correct this.

Our approach is to extend the pipeline and do a matching, whereby we replace the `lemma` form of "z. B." to the appropriate long form.

**IMPORTANT** design principle by SpaCy is, that one **always keeps the possibility to restore the original text**, so we are **NOT to modify `token.text`**. In the analysed form, we can do whatever we want.

It is typical to add layers to the pipeline which modify the analysis.

For our purposes, we will use rule based matching to achieve our goals.

A detailed description on rule based matching in SpaCy can be found [here](https://spacy.io/usage/rule-based-matching), or [here](https://medium.com/@ashiqgiga07/rule-based-matching-with-spacy-295b76ca2b68)

### Build the matcher

With the help of rule based matching we create a matcher that reacts to the presence of "z. B." exactly, then we use this matcher to define a pipeline step, that after matching, replaces the lemmas of the tokens "z." and "B." to  their full written equivalent.  

In [57]:
zb_matcher = Matcher(nlp.vocab, validate=True) # Please instantiate a matcher with the appropriate parameters - think about all the words of the corpus...
pattern = [{"LOWER": "z."}, {"TEXT": "B."}] 

zb_matcher.add("z.B. pattern", None, pattern) # Please add an appropriate pattern to the matcher to match "z. B."

def zb_replacer(doc):
    matched_spans = []
    # Please use the matcher to get matches!
    matches = zb_matcher(doc)
    # Plsease iterate over the matches!
    for match_id, start, end in matches:
        span = doc[start:end] # get the span of text based on the matches coordinates!
        matched_spans.append(span)
        print("ZB MATCH!!!")

    # Please iterate over matched spans
    for i in matched_spans:  
        # And replace their lemmas to the appropriate ones!
        # Please observe, that you don't have the ID of the desired lemmas, just the their string form.
        i[0].lemma_ = 'zum'
        i[1].lemma_ = 'Beispiel'
    return doc

### Register it to the pipeline

After creating this processing step, we register it to be part of the pipeline and then run our analysis again.

In [58]:
# Plase register the new zb_replacer to the pipeline!
# Think about, where to place it!
nlp.add_pipe(zb_replacer) 

### Re-do the analysis and observe results

In [59]:
doc=nlp(text)

ZB MATCH!!!


In [60]:
print_tokens_for_sentence(doc,-1)

-------------------  -------------------  -----
Außerdem             Außerdem             ADV
ist                  sein                 AUX
sie                  ich                  PRON
eine                 einen                DET
Minderheitensprache  Minderheitensprache  NOUN
in                   in                   ADP
einigen              einig                DET
europäischen         europäisch           ADJ
und                  und                  CCONJ
außereuropäischen    außereuropäisch      ADJ
Ländern              Land                 NOUN
,                    ,                    PUNCT
z.                   zum                  ADP
B.                   Beispiel             NOUN
in                   in                   ADP
Rumänien             Rumänien             PROPN
und                  und                  CCONJ
Südafrika            Südafrika            PROPN
,                    ,                    PUNCT
sowie                sowie                CCONJ
Nationalsprach

## What are those ugly pronunciation signs doing there?

OK, so far so good. Let's observe, what is the problem with the first sentence!

In [61]:
doc=nlp(text)

ZB MATCH!!!


In [62]:
print_tokens_for_sentence(doc,0)


---------  ---------------  -----
Die        der              DET
deutsche   deutsch          ADJ
Sprache    Sprache          NOUN
bzw.       beziehungsweise  CCONJ
Deutsch    Deutsch          NOUN
(          (                PUNCT
[          [                NOUN
dɔʏ̯t͡ʃ      dɔʏ̯t͡ʃ            PROPN
]          ]                ADP
;          ;                PUNCT
abgekürzt  abkürzen         VERB
dt         dt               X
.          .                PUNCT
---------  ---------------  -----


As we can see, poor pipeline can not really cope with the pronunciation markings of the phonetic alphabet, and thus thinks, that the signs are representing a foreign proper noun. 

We would like to remedy this, and since we do expect further texts from the corpus to contain these inserted phonetics, we would like to match, merge and replace.

## Building up matcher for PRONUNCIATION

To be more specific, we again first build up a matcher, that aims at the "square brackets" markings around the pronunciation. The task is to match everything between square brackets, or to be more specific: **everything that starts with an opening square bracket, and finishes with ";"**.

This matcher can then be used to:

1. Merge the resulting matching `span` into one token
2. Replace the token's lemma to "PRONUNCIATION"

For this to be achievable, we have to first register "PRONUNCIATION" as part of the vocabulary, moreover mark it as ["stopword"](https://en.wikipedia.org/wiki/Stop_words). (More on SpaCy's stopword handling [here](https://medium.com/@makcedward/nlp-pipeline-stop-words-part-5-d6770df8a936)) See below.

In [63]:
# Please instantiate and build the matcher as before with the appropriate pattern!
# Make it so, that the pattern will match ALL future pronunciations, not just the present one!
pronunciation_matcher = Matcher(nlp.vocab, validate=True) 
pattern = [{"IS_PUNCT": True},
           {"IS_LOWER": True},
           {"IS_PUNCT": True},
           {"IS_PUNCT": True}]
pronunciation_matcher.add("pronunciation pattern", None, pattern)

# We set the properties for the new word "PRONUNCIATION"
lex = nlp.vocab['PRONUNCIATION']
lex.is_oov = False
lex.is_stop = True

def pronunciation_replacer(doc):
    
    # Using the template above, please build a pronunciation replacer, that

    # 1. gets the matches
    # 2. merges them into one
    matched_spans = []
    matches = pronunciation_matcher(doc)
    for match_id, start, end in matches:
      span = doc[start:end] 
      matched_spans.append(span)
      print("Pronunciation Match")
    
    # 3. Replaces their lemma string and lemma ID
    # 4. sets it's POS to "NOUN"
    merged_span = span.merge()
    merged_span.lemma_ = "PRONUNCIATION"
    merged_span.pos_ = "NOUN"
    
    return doc


nlp.add_pipe(pronunciation_replacer, after="zb_replacer") 

### Observing result

In [64]:
doc=nlp(text)
print_tokens_for_sentence(doc,0)

ZB MATCH!!!
Pronunciation Match
---------  ---------------  -----
Die        der              DET
deutsche   deutsch          ADJ
Sprache    Sprache          NOUN
bzw.       beziehungsweise  CCONJ
Deutsch    Deutsch          NOUN
(          (                PUNCT
[dɔʏ̯t͡ʃ];   PRONUNCIATION    NOUN
abgekürzt  abkürzen         VERB
dt         dt               X
.          .                PUNCT
---------  ---------------  -----


In the future, we decide, we would not want to include the pronunciation tokens in our view. So we have to mark them as wtopwords.

### Registering PRONUNCIATION as a stopword

Stopwords are typically those words, which do not contribute to the meaning of the sentence, are just there for syntactic reasons. There is a vague running list of these for languages. We will use and extend the German one in SpaCy.


In [65]:
# import stop words from GERMAN language data
from spacy.lang.de.stop_words import STOP_WORDS

# Add PRONUNCIATION to stopwords
STOP_WORDS.add("PRONUNCIATION")

In [66]:
nlp.pipeline

[('tagger', <spacy.pipeline.pipes.Tagger at 0x7f85b9e96f98>),
 ('parser', <spacy.pipeline.pipes.DependencyParser at 0x7f85b43316a8>),
 ('ner', <spacy.pipeline.pipes.EntityRecognizer at 0x7f85b4331708>),
 ('zb_replacer', <function __main__.zb_replacer>),
 ('pronunciation_replacer', <function __main__.pronunciation_replacer>)]

But since we will only be able to manipulate the lemmas of the pronunciation markings, we would have to let SpaCy know, that - in contrast to the default behavior, where stopwords are filtered on `text` level, we would like to have a new property for words, that is based on `lemma` level stopword filtering.

For these we will use extensions!

For more info please see [here](https://spacy.io/api/token#set_extension)!

In [67]:
from spacy.tokens import Token

# Please define a function (or lambda expression!) that checks if a Token, or its lower case for, 
# OR it's lemma string is contained it he stopword list above.
stop_words_getter = lambda token: token.is_stop or token.lower_ in STOP_WORDS or token.lemma_ in STOP_WORDS

# Set the above defined function as a extension for Token under the name "is_lemma_stop" as a getter!
Token.set_extension('is_lemma_stop', getter=stop_words_getter, force=True)

In [68]:
doc=nlp(text)

ZB MATCH!!!
Pronunciation Match


In [69]:
print_tokens_for_sentence(doc,0, stopwords=True)

assert len(list(doc.sents)[0]) == 10

---------  ---------------  -----
deutsche   deutsch          ADJ
Sprache    Sprache          NOUN
bzw.       beziehungsweise  CCONJ
Deutsch    Deutsch          NOUN
(          (                PUNCT
abgekürzt  abkürzen         VERB
dt         dt               X
.          .                PUNCT
---------  ---------------  -----


## Language detection

We could also observe, that there is some English text inbetween our nice German sentences. We would like to detect foreign sentences and by later processing, ignore / skip them.

For this to be achievable, we need some language detection capabilities.

Luckily enough, we can make it part of our pipeline via [this extension](#https://spacy.io/universe/project/spacy-langdetect).

### Standard installation

In [70]:
%%capture
!pip install spacy-langdetect

In [71]:
#Please import the language detector!
from spacy_langdetect import LanguageDetector

### Adding language detection to our pipeline

In [72]:
# Please register it to the pipeline as the final step of processing!
nlp.add_pipe(LanguageDetector(), name='language_detector', last=True)

In [73]:
nlp.pipeline

[('tagger', <spacy.pipeline.pipes.Tagger at 0x7f85b9e96f98>),
 ('parser', <spacy.pipeline.pipes.DependencyParser at 0x7f85b43316a8>),
 ('ner', <spacy.pipeline.pipes.EntityRecognizer at 0x7f85b4331708>),
 ('zb_replacer', <function __main__.zb_replacer>),
 ('pronunciation_replacer', <function __main__.pronunciation_replacer>),
 ('language_detector',
  <spacy_langdetect.spacy_langdetect.LanguageDetector at 0x7f85b828c438>)]

### Observing results

In [74]:
doc = nlp(text)

ZB MATCH!!!
Pronunciation Match


In [75]:
attribs = []
for sentence in doc.sents:
    attribs.append([list(sentence)[:15],"...", sentence._.language])
print(tabulate(attribs))

# Please observe how one accesses anextension!!

-----------------------------------------------------------------------------------------------------------------------------  ---  -----------------------------------------------
[Die, deutsche, Sprache, bzw., Deutsch, (, [dɔʏ̯t͡ʃ];, abgekürzt, dt, .]                                                         ...  {'language': 'de', 'score': 0.9999982539775079}
[oder, dtsch, ., ), ist, eine, westgermanische, Sprache, .,                                                                    ...  {'language': 'de', 'score': 0.9999940650974677}

]
[And, this, is, an, English, sentence, inbetween, .,                                                                           ...  {'language': 'en', 'score': 0.9999972134255091}

]
[Ihr, Sprachraum, umfasst, Deutschland, ,, Österreich, ,, die, Deutschschweiz, ,, Liechtenstein, ,, Luxemburg, ,, Ostbelgien]  ...  {'language': 'de', 'score': 0.9999962650559461}
[Außerdem, ist, sie, eine, Minderheitensprache, in, einigen, europäischen, und, außereuropäi

# Creating final generator for cleaned text

Typically for a later stage of NLP, we would like to have a generator like function, which allows us to iteratively access the corpus, albeit in it's cleaned and encoded form. Integer encoding (as well as one hot encoding) are quite typical representations of text.

In this spirit, we would like to implement a generator, that gives back an **array of lemmas OR lemma IDs for each sentence in the corpus, filtering out non-German sentences and punctuation / space marks**. 

In [76]:
# Please implement a generator function that yields the text of the corpus as lists of sentences
# Based on the parameters either as a list of strings or a list of IDs
# It should filter out non-German sentences
# as well as topwords based on lemmas
# and punctuation and "space like" characters!

def sentence_generator(doc, ids=False):
  stopwords = True
  
  for sentence in doc.sents:
    # Create a list that stores token.lemma_
    list_token_lemma=[]
    # Create a list that stores token.lemma (ids)
    list_token_lemma_ids=[]
    #go forward only if language of sentence is German
    if sentence._.language['language'] =="de":
      # iterate over the tokens in the sentence, so that we can
      # remove unwanted tokens
      for token in list(sentence):
        # if we have stopwords on lemma level(extentions) and text level, PASS
        if stopwords and token._.is_lemma_stop:
          pass
        # if POS is not a PUNCTUATION and its not SPACE, append the 
        # respective lists for lemma and lemma ids
        elif token.pos_ != 'PUNCT' and token.pos_ != 'SPACE':
          list_token_lemma.append(token.lemma_)
          list_token_lemma_ids.append(token.lemma)
      
      if ids == True:
        yield list_token_lemma_ids
      else:
        yield list_token_lemma
        

We had 5 sentenences out of which 1 was english which should be removed. 

In [77]:
for i in sentence_generator(doc):
    print(i,"\n")

for i in sentence_generator(doc, ids=True):
    print(i,"\n")


['deutsch', 'Sprache', 'beziehungsweise', 'Deutsch', 'abkürzen', 'dt'] 

['dtsch', 'westgermanische', 'Sprache'] 

['Sprachraum', 'umfasst', 'Deutschland', 'Österreich', 'Deutschschweiz', 'Liechtenstein', 'Luxemburg', 'Ostbelgien', 'Südtirol', 'Elsass', 'Lothringen', 'Nordschleswig'] 

['Minderheitensprache', 'europäisch', 'außereuropäisch', 'Land', 'Beispiel', 'Rumänien', 'Südafrika', 'Nationalsprache', 'afrikanisch', 'Namibia'] 

[5968319817064592459, 8431935777423264011, 16143637279988465102, 13347145995516113707, 12068858602874567954, 5135506797272647618] 

[2552743035069842888, 7654685629011980891, 8431935777423264011] 

[11854469037278879099, 7289263729939212449, 3491614202785599281, 16047064563126251420, 3469156011154928224, 10833980334450146958, 15216956676957942053, 14493420987399493547, 14425170055224073740, 14854674721094831692, 5682654018506929560, 10694615845175474381] 

[13853446524293058697, 512110525822973470, 15751849195492229329, 731233208058718707, 176351906757609250